In [1]:
# Import modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
root_dir = os.path.expanduser("~/work/data/")
csv_file = os.path.join(root_dir, 'raw-data/outagesExample.csv')
# check if file exists
if not os.path.isfile(csv_file):
    raise FileNotFoundError(f"The file {csv_file} does not exist.")
outages_df = pd.read_csv(csv_file)
outages_df.head()


,number,priority,outage_start_time,assignment_group,configuration_item,service_category,outage_end_time
0,ISSUE0006826,2,2025-12-19 12:03:49,group_1,system_1,service_1,2025-12-19 12:16:20
1,ISSUE0006825,2,2025-12-18 15:10:56,group_2,system_2,service_2,2025-12-18 16:28:59
2,ISSUE0006820,2,2025-12-16 23:50:52,group_3,system_3,service_1,2025-12-17 00:14:59
3,ISSUE0006817,2,2025-12-15 17:00:00,group_4,system_4,service_2,2025-12-16 11:49:36
4,ISSUE0006816,1,2025-12-16 10:19:53,group_5,system_5,service_3,2025-12-16 11:26:55


In [3]:
# create new data frame with converted datetime columns and outage duration in minutes
outages_converted_df = outages_df.copy()
outages_converted_df['outage_start_time'] = pd.to_datetime(outages_converted_df['outage_start_time'])
outages_converted_df['outage_end_time'] = pd.to_datetime(outages_converted_df['outage_end_time'])
outages_converted_df['outage_duration_minutes'] = (outages_converted_df['outage_end_time'] - outages_converted_df['outage_start_time']).dt.total_seconds() / 60
outages_converted_df.head()

# list earliest and latest outage start times
earliest_outage = outages_converted_df['outage_start_time'].min()
latest_outage = outages_converted_df['outage_start_time'].max()
print(f"Earliest outage start time: {earliest_outage}")
print(f"Latest outage start time: {latest_outage}")

Earliest outage start time: 2024-07-01 07:02:00
Latest outage start time: 2025-12-19 12:03:49


In [4]:
# Create a calendar for fiscal years and weeks where fiscal year starts on july 1 each year and ends on june 30 the following year, there are only 52 weeks in a fiscal year, and weeks start on mondays starting in 2024 to current year
fiscal_calendar = pd.date_range(start='2024-07-01', end=pd.Timestamp.today(), freq='W-MON')
fiscal_calendar_df = pd.DataFrame({'date': fiscal_calendar})
fiscal_calendar_df['fiscal_year'] = fiscal_calendar_df['date'].dt.year + np.where(fiscal_calendar_df['date'].dt.month >= 7, 1, 0)

# add fiscal week column where fiscal week starts on july 1 each year
fiscal_calendar_df['fiscal_week'] = fiscal_calendar_df.groupby('fiscal_year').cumcount() + 1
fiscal_calendar_df['fiscal_week'] = 'W' + fiscal_calendar_df['fiscal_week'].astype(str).str.zfill(2)

# add fiscal month column where fiscal month starts on july 1 each year
fiscal_calendar_df['fiscal_month'] = ((fiscal_calendar_df['date'].dt.month - 7) % 12) + 1
fiscal_calendar_df['fiscal_month'] = 'M' + fiscal_calendar_df['fiscal_month'].astype(str).str.zfill(2)

#add fiscal quarter column where fiscal quarter starts on july 1 each year, there are only 4 fiscal quarters in a fiscal year values should be Q1 starting in July, Q2 starting in October, Q3 starting in January, Q4 starting in April
fiscal_calendar_df['fiscal_quarter'] = ((fiscal_calendar_df['date'].dt.month - 7) // 3) % 4 + 1
fiscal_calendar_df['fiscal_quarter'] = 'Q' + fiscal_calendar_df['fiscal_quarter'].astype(str)

# export fiscal calendar dataframe to csv
output_csv_file = os.path.join(root_dir, 'parsed-data/fiscal_calendar.csv')
fiscal_calendar_df.to_csv(output_csv_file, index=False)
print(f"Fiscal Calendar dataframe exported to {output_csv_file}")

fiscal_calendar_df.head()

Fiscal Calendar dataframe exported to /home/jovyan/work/data/parsed-data/fiscal_calendar.csv


,date,fiscal_year,fiscal_week,fiscal_month,fiscal_quarter
0,2024-07-01,2025,W01,M01,Q1
1,2024-07-08,2025,W02,M01,Q1
2,2024-07-15,2025,W03,M01,Q1
3,2024-07-22,2025,W04,M01,Q1
4,2024-07-29,2025,W05,M01,Q1


In [10]:
# Merge fiscal calendar with outage converted dataframe to get fiscal year and week for each outage based on outage start time
fc_outage_merge_df = pd.merge_asof(
    outages_converted_df.sort_values('outage_start_time'),
    fiscal_calendar_df.sort_values('date'),
    left_on='outage_start_time',
    right_on='date',
    direction='backward'
)
# dropping redundant date column from fiscal calendar
fc_outage_merge_df = fc_outage_merge_df.drop(columns=['date'])
fc_outage_merge_df.head()

# export merged dataframe to csv
output_csv_file = os.path.join(root_dir, 'parsed-data/outages_with_fiscal_info.csv')
fc_outage_merge_df.to_csv(output_csv_file, index=False)
print(f"Merged dataframe exported to {output_csv_file}")

fc_outage_merge_df.head()

Merged dataframe exported to /home/jovyan/work/data/parsed-data/outages_with_fiscal_info.csv


,number,priority,outage_start_time,assignment_group,configuration_item,service_category,outage_end_time,outage_duration_minutes,fiscal_year,fiscal_week,fiscal_month,fiscal_quarter
0,ISSUE0005178,1,2024-07-01 07:02:00,group_49,system_48,service_2,2024-07-01 18:02:00,660.000000,2025,W01,M01,Q1
1,ISSUE0005219,2,2024-07-01 11:03:41,group_7,system_7,service_10,2024-07-02 08:14:00,1270.316667,2025,W01,M01,Q1
2,ISSUE0005257,2,2024-07-12 13:35:21,group_48,system_152,service_17,2024-08-14 09:44:45,47289.400000,2025,W02,M01,Q1
3,ISSUE0005255,2,2024-07-15 06:05:41,group_14,system_153,service_2,2024-07-15 08:55:54,170.216667,2025,W03,M01,Q1
4,ISSUE0005291,2,2024-07-23 07:39:00,group_4,system_135,service_10,2024-07-29 13:01:00,8962.000000,2025,W04,M01,Q1


In [6]:
# Function to calculate MTBF for a given time period where system is expected 24/7 operational time and MTBF is calculated as total operational time divided by number of outages
def calculate_mtbf(df, time_period_start, time_period_end):
    # Filter outages within the time period
    mask = (df['outage_start_time'] >= time_period_start) & (df['outage_start_time'] < time_period_end)
    filtered_df = df.loc[mask]
    
    # Calculate total operational time in minutes
    total_time_minutes = (time_period_end - time_period_start).total_seconds() / 60
    
    # Calculate number of outages
    num_outages = len(filtered_df)
    
    # Calculate MTBF in minutes
    if num_outages == 0:
        return total_time_minutes  # No outages, MTBF is total time
    else:
        mtbf_minutes = total_time_minutes / num_outages
        return mtbf_minutes
    
    # Calculate MTBF in hours
    mtbf_hours = mtbf_minutes / 60
    return mtbf_hours

# Example: Calculate MTBF for fiscal year 2025 (July 1, 2024 to June 30, 2025)
fy_start = pd.Timestamp('2024-07-01')
fy_end = pd.Timestamp('2025-07-01')
mtbf_fy2025 = calculate_mtbf(fc_outage_merge_df, fy_start, fy_end)
print(f"MTBF for Fiscal Year 2025: {mtbf_fy2025:.2f} hours")

# Display mtbf in dd hh:mm:ss format
def format_mtbf(mtbf_minutes):
    total_seconds = int(mtbf_minutes * 60)
    days, remainder = divmod(total_seconds, 86400)
    hours, remainder = divmod(remainder, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{days}d {hours:02}:{minutes:02}:{seconds:02}"

print(f"MTBF for FY25 (dd hh:mm:ss): {format_mtbf(mtbf_fy2025)}")

MTBF for Fiscal Year 2025: 2810.70 hours
MTBF for FY25 (dd hh:mm:ss): 1d 22:50:41


In [7]:
# Function to calculate difference between two MTBF values and express as percentage improvement or degradation improvement or degradation is calculated as 0 then display as 0%
def calculate_mtbf_difference(mtbf_old, mtbf_new):
    if mtbf_old == 0:
        return 0.0
    difference = mtbf_new - mtbf_old
    percentage_change = (difference / mtbf_old) * 100
    return percentage_change

# example usage of calculate_mtbf_difference function
mtbf_fy2024 = 2500  # example previous MTBF in hours
mtbf_difference = calculate_mtbf_difference(mtbf_fy2024, mtbf_fy2025)
diff_list = ['improvement' if mtbf_difference > 0 else 'degradation' if mtbf_difference < 0 else 'no change']
print(f"MTBF difference from FY24 to FY25: {mtbf_difference:.2f}% ({diff_list[0]}) from {format_mtbf(mtbf_fy2024)} hours to {format_mtbf(mtbf_fy2025)} hours" )

MTBF difference from FY24 to FY25: 12.43% (improvement) from 1d 17:40:00 hours to 1d 22:50:41 hours


In [8]:

fy_weeks_outage_count = fc_outage_merge_df.groupby(['fiscal_year', 'fiscal_week']).size().reset_index(name='outage_count')
fy_weeks_outage_count.head(15)

,fiscal_year,fiscal_week,outage_count
0,2025,W01,2
1,2025,W02,1
2,2025,W03,1
3,2025,W04,4
4,2025,W05,1
5,2025,W06,3
6,2025,W07,3
7,2025,W08,2
8,2025,W09,8
9,2025,W10,2


In [9]:
# get mtbf per fiscal week for fiscal year 2025
fy2025_weeks = fy_weeks_outage_count[fy_weeks_outage_count['fiscal_year'] == 2025]
fy2025_weeks['mtbf_minutes'] = ((7 * 24 * 60) / fy2025_weeks['outage_count']).replace(np.inf, 7 * 24 * 60)
fy2025_weeks['mtbf_formatted'] = fy2025_weeks['mtbf_minutes'].apply(format_mtbf)


# get mtbf per fiscal week for fiscal year 2026
fy2026_weeks = fy_weeks_outage_count[fy_weeks_outage_count['fiscal_year'] == 2026]
fy2026_weeks['mtbf_minutes'] = ((7 * 24 * 60) / fy2026_weeks['outage_count']).replace(np.inf, 7 * 24 * 60)
fy2026_weeks['mtbf_formatted'] = fy2026_weeks['mtbf_minutes'].apply(format_mtbf)

fy2025_fy2026_weeks_merged = pd.merge(
    fy2025_weeks,
    fy2026_weeks,
    on='fiscal_week',
    suffixes=('_fy2025', '_fy2026')
)
fy2025_fy2026_weeks_merged['mtbf_difference_percent'] = fy2025_fy2026_weeks_merged.apply(
    lambda row: calculate_mtbf_difference(row['mtbf_minutes_fy2025'], row['mtbf_minutes_fy2026']), axis=1
)

# drop fiscal year columns
fy2025_fy2026_weeks_merged = fy2025_fy2026_weeks_merged.drop(columns=['fiscal_year_fy2025', 'fiscal_year_fy2026'])
fy2025_fy2026_weeks_merged

/tmp/ipykernel_517/753041947.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fy2025_weeks['mtbf_minutes'] = ((7 * 24 * 60) / fy2025_weeks['outage_count']).replace(np.inf, 7 * 24 * 60)
/tmp/ipykernel_517/753041947.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fy2025_weeks['mtbf_formatted'] = fy2025_weeks['mtbf_minutes'].apply(format_mtbf)
/tmp/ipykernel_517/753041947.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,fiscal_week,outage_count_fy2025,mtbf_minutes_fy2025,mtbf_formatted_fy2025,outage_count_fy2026,mtbf_minutes_fy2026,mtbf_formatted_fy2026,mtbf_difference_percent
0,W01,2,5040.0,3d 12:00:00,4,2520.0,1d 18:00:00,-50.000000
1,W02,1,10080.0,7d 00:00:00,3,3360.0,2d 08:00:00,-66.666667
2,W03,1,10080.0,7d 00:00:00,4,2520.0,1d 18:00:00,-75.000000
3,W04,4,2520.0,1d 18:00:00,4,2520.0,1d 18:00:00,0.000000
4,W05,1,10080.0,7d 00:00:00,7,1440.0,1d 00:00:00,-85.714286
5,W06,3,3360.0,2d 08:00:00,4,2520.0,1d 18:00:00,-25.000000
6,W07,3,3360.0,2d 08:00:00,6,1680.0,1d 04:00:00,-50.000000
7,W08,2,5040.0,3d 12:00:00,3,3360.0,2d 08:00:00,-33.333333
8,W09,8,1260.0,0d 21:00:00,7,1440.0,1d 00:00:00,14.285714
9,W10,2,5040.0,3d 12:00:00,6,1680.0,1d 04:00:00,-66.666667
